import json
import ast
import os


directory = '../../data/cache/2020-05-12_13-42-02'
master = {}
master['apps'] = []

for file in os.listdir(directory):
    if file.endswith(".txt"):
        cache_file = os.path.join(directory, file)

        with open(cache_file, encoding='utf8') as f:
            for line in f:
                app_details = ast.literal_eval(line)
                master['apps'].append(app_details)

with open(directory+'/steamstore.json','w') as out:
    json.dump(master,out)

In [19]:
import pandas as pd
import json

In [343]:
data_path = '../../data/cache/store_steamspy/apps.json'

with open(data_path, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

app_data = pd.DataFrame(raw_data['apps'])

## Feature Engineering

already_released y/n
Release date


In [344]:
app_data.drop('supported_languages',axis=1,inplace=True)

In [345]:
app_data.drop('recommendations',axis=1, inplace=True)

In [346]:
def get(element,key):
    try:
        return element[key]
    except:
        return None

In [347]:
app_data['price'] = app_data['price_overview'].apply(lambda x: get(x,'initial'))
#app_data.drop('price_overview',axis=1,inplace=True)

In [348]:
app_data['released'] = app_data['release_date'].apply(lambda x: not get(x,'coming_soon'))
app_data['release_date'] = app_data['release_date'].apply(lambda x: get(x, 'date'))


In [349]:
app_data.drop('genres',axis=1,inplace=True)

In [350]:
def get_owners(string):
    bounds = [int(item.replace(',','')) for item in string.split(' .. ')]
    return int((bounds[0]+bounds[1])/2)

In [351]:
app_data['median_owners'] = app_data['owners'].apply(get_owners)

In [352]:
app_data['categories'] = app_data['categories'].apply(lambda x: [{'id': 2, 'description': 'Single-player'}] if x is None else x)

In [353]:
app_data['categories'] = app_data['categories'].apply(lambda tags: [tag['description'] for tag in tags])

In [354]:
app_data.dropna(subset=['languages','developers'],inplace=True)

In [355]:
app_data['price'] = app_data.apply((lambda x: 0 if x['is_free']==True else x['price']), axis=1)